In [1]:
run -i 'test_dbi_evolution_oracles.py'

[Qibo 0.2.5|INFO|2024-02-08 04:16:31]: Using qibojit (numba) backend on /CPU:0
[Qibo 0.2.5|WARNING|2024-02-08 04:16:31]: Calculating the dense form of a symbolic Hamiltonian. This operation is memory inefficient.


ZX0.1
3 0.5264075362294286
6 0.12267264894892986
12 0.03018952498939946
<class 'qibo.models.circuit.Circuit'>
q0:     ───U─U─────U─U─────U─U─────U─U─────U─U─────U─U─────U─U─────U─U─────U─U ...
q1:     ─U─U─U─U─U─U─U─U─U─U─U─U─U─U─U─U─U─U─U─U─U─U─U─U─U─U─U─U─U─U─U─U─U─U─U ...
q2:     ─U─────U─U─────U─U─────U─U─────U─U─────U─U─────U─U─────U─U─────U─U──── ...

q0: ... ─────U─U─────U─U─────U─U───
q1: ... ─U─U─U─U─U─U─U─U─U─U─U─U─U─
q2: ... ─U─U─────U─U─────U─U─────U─
3.651778047740365
3.651778047740365


In [39]:
from qibo.hamiltonians import SymbolicHamiltonian
from qibo import symbols
from double_bracket_evolution_oracles import *
from group_commutator_iteration_transpiler import *

"""Test create evolution oracle"""

h_input = SymbolicHamiltonian( symbols.Z(0)+symbols.X(0) + symbols.Z(0) * symbols.X(1) + symbols.Y(2) + symbols.Y(1) * symbols.Y(2), nqubits = 3 )

## Test initialization of evolution oracles
#By default EvolutionOracle initializes with text_strings oracle type
input_hamiltonian_evolution_oracle_text_strings = EvolutionOracle(h_input, "ZX", mode_evolution_oracle = EvolutionOracleType.text_strings)

c1 = input_hamiltonian_evolution_oracle_text_strings.circuit(0.1)
if input_hamiltonian_evolution_oracle_text_strings.mode_evolution_oracle is EvolutionOracleType.text_strings:
    assert isinstance( c1, str), "Should be a string here"
print( c1 )

# Initialize with EvolutionOracleType hamiltonian_simulation
input_hamiltonian_evolution_oracle_hamiltonian_simulation = EvolutionOracle(h_input, "ZX",
                                 mode_evolution_oracle = EvolutionOracleType.hamiltonian_simulation)
input_hamiltonian_evolution_oracle_hamiltonian_simulation.please_be_verbose = True
c2 = input_hamiltonian_evolution_oracle_hamiltonian_simulation.circuit(2)
assert np.linalg.norm( c2.unitary() - input_hamiltonian_evolution_oracle_hamiltonian_simulation.h.exp(2) ) < 1

##For some reason running this makes the norm check below blow up to large value
# if input_hamiltonian_evolution_oracle_hamiltonian_simulation.mode_evolution_oracle is EvolutionOracleType.hamiltonian_simulation:
#     print(type(c2))
#     assert isinstance( c2, type(h_input.circuit(0.1)) ), "Should be a qibo.Circuit here"
# print(c2.draw())


# Initialize with EvolutionOracleType numerical
input_hamiltonian_evolution_oracle_numerical = EvolutionOracle(h_input, "ZX",
                                                                mode_evolution_oracle = EvolutionOracleType.numerical )

c3 = input_hamiltonian_evolution_oracle_numerical.circuit(2)
if input_hamiltonian_evolution_oracle_numerical.mode_evolution_oracle is EvolutionOracleType.numerical:
    assert isinstance( c3, type(h_input.exp(0.1)) ), "Should be a np.array here"
print(np.linalg.norm( c2.unitary() - c3 ))


[Qibo 0.2.5|WARNING|2024-02-08 04:26:52]: Calculating the dense form of a symbolic Hamiltonian. This operation is memory inefficient.


ZX0.1
3 0.43947054352187015
6 0.10267893936993515
12 0.025341696660493532
0.02534169666049345


In [62]:
from double_bracket import *

dbi = DoubleBracketIteration(deepcopy(h_input.dense))
dbi.mode = DoubleBracketGeneratorType.group_commutator

In [63]:
dbi.off_diagonal_norm

32.0

In [64]:
d_0 = SymbolicHamiltonian(symbols.Z(0) * symbols.Z(1) + symbols.Z(2)+symbols.Z(0), nqubits = 3 )

In [65]:
dbi(0.51, d = d_0.dense)
dbi.off_diagonal_norm

[Qibo 0.2.5|WARNING|2024-02-08 04:32:15]: Calculating the dense form of a symbolic Hamiltonian. This operation is memory inefficient.


TypeError: Hamiltonian.exp() missing 1 required positional argument: 'a'

In [56]:
## Test more fancy functionalities
input_hamiltonian_evolution_oracle_hamiltonian_simulation.please_be_verbose = False
gci = GroupCommutatorIterationWithEvolutionOracles( deepcopy(input_hamiltonian_evolution_oracle_hamiltonian_simulation ))


query_list = gci.group_commutator_query_list( 0.2, d_0, input_hamiltonian_evolution_oracle_hamiltonian_simulation )
gci(0.51, d_0)
u_dagger = gci.iterated_hamiltonian_evolution_oracle.before_circuit.unitary()

dbi_from_gci  = u_dagger @ h_input.matrix @ u_dagger.T.conj()

In [59]:
dbi_from_gci - dbi.h.matrix

matrix([[ 0.37873495-7.23646444e-17j, -0.61455831+6.61914883e-01j,
          1.65379471-4.82329292e-01j, -1.96431484+8.48346097e-01j,
         -0.66648988-6.34914954e-01j,  0.72872705+1.92520016e-01j,
          0.46450583-3.48735459e-01j, -0.34937171+7.29602858e-01j],
        [-0.61455831-6.61914883e-01j,  1.3880865 +1.65819142e-16j,
          1.7100767 -2.63880526e-01j,  1.22172438-4.00032815e-01j,
         -1.71509717-2.99995268e-01j,  0.06517562-7.21879491e-02j,
          0.05236784-8.00791699e-01j,  0.37782571+6.27793798e-02j],
        [ 1.65379471+4.82329292e-01j,  1.7100767 +2.63880526e-01j,
          0.9379625 -9.61058031e-17j, -0.31832345+1.01913826e+00j,
          0.50572804-3.07643806e-01j, -0.60181522-3.91004322e-02j,
          0.33658616+6.18667086e-01j, -0.44629897+6.13539951e-02j],
        [-1.96431484-8.48346097e-01j,  1.22172438+4.00032815e-01j,
         -0.31832345-1.01913826e+00j,  3.00438693-1.10863074e-16j,
         -0.45113968-1.38908374e+00j,  0.32741828+2.4352710

In [43]:
dbi(0.2, d = d_0.dense)
np.linalg.norm(dbi.h.matrix - h_input.dense.matrix)

[Qibo 0.2.5|WARNING|2024-02-08 04:27:03]: Calculating the dense form of a symbolic Hamiltonian. This operation is memory inefficient.


3.626108312062859

In [22]:
from functools import reduce
before_circuit = reduce(Circuit.__add__,  query_list['backwards'])
after_circuit = reduce(Circuit.__add__,  query_list['forwards'])
print( before_circuit.draw() )
print( after_circuit.draw() )

u_before = before_circuit.unitary()
u_after = after_circuit.unitary()

print( np.linalg.norm( u_before @ input_hamiltonian_evolution_oracle_hamiltonian_simulation.h.matrix@ u_after -gci.h.matrix ))
frame_shifted_input_hamiltonian_evolution_oracle = FrameShiftedEvolutionOracle( 
        input_hamiltonian_evolution_oracle_hamiltonian_simulation, 'Step 1', before_circuit, after_circuit)

print( np.linalg.norm( gci.h.exp(0.3) - frame_shifted_input_hamiltonian_evolution_oracle.circuit(0.3).unitary()))

q0: ─U─────U───U─U─────U─U─────U─U───U─────U───U─U─────U─U─────U─U───
q1: ─U─────U─U─U─U─U─U─U─U─U─U─U─U─U─U─────U─U─U─U─U─U─U─U─U─U─U─U─U─
q2: ───U─U───U─────U─U─────U─U─────U───U─U───U─────U─U─────U─U─────U─
q0: ───U─U─────U─U─────U─U───U─────U───U─U─────U─U─────U─U───U─────U─
q1: ─U─U─U─U─U─U─U─U─U─U─U─U─U─────U─U─U─U─U─U─U─U─U─U─U─U─U─U─────U─
q2: ─U─────U─U─────U─U─────U───U─U───U─────U─U─────U─U─────U───U─U───
9.519346834291838
4.144980851654998
